In [1]:
import pandas as pd

import quandl
import my_secrets
quandl.ApiConfig.api_key = my_secrets.QUANDL_API_KEY

import random
import numpy as np

from scipy import stats



import altair as alt
if not alt.data_transformers.active == 'json':  # Check json isn't already active
    alt.data_transformers.enable('json')
alt.renderers.enable('notebook')



%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

## Finding S&P 500 dataset
- The code Below downloads the S&P 500 data from Quandl.
- After that we are going to compute the percentage change on a daily basis as a value between 0 and 100% - not a decimal.

In [2]:
sp500 = quandl.get("CHRIS/CME_SP1")

sp500_sample = sp500.loc["2015":"2017"].dropna()

In [3]:
sp500_sample.head()

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,
2015-01-02,2055.0,2067.0,2039.5,2046.0,6.1,2046.3,5704.0,123911.0
2015-01-05,2043.7,2048.1,2010.0,2016.2,30.4,2015.9,11355.0,124285.0
2015-01-06,2018.9,2023.2,1984.0,1994.9,21.5,1994.4,11845.0,125744.0
2015-01-07,1997.1,2024.0,1995.8,2020.0,25.2,2019.6,4685.0,122933.0
2015-01-08,2021.2,2058.3,2020.8,2054.5,35.4,2055.0,9723.0,123385.0


In [4]:
market = sp500_sample['Last'].pct_change().dropna() * 100 #Last is the closing price of shares.
market.head()

Date
2015-01-05   -1.456500
2015-01-06   -1.056443
2015-01-07    1.258208
2015-01-08    1.707921
2015-01-09   -0.934534
Name: Last, dtype: float64

## Finding IBM dataset
- The code Below downloads the IBM data from Quandl.
- After that we are going to compute the percentage change on a daily basis as a value between 0 and 100% - not a decimal.

In [5]:
IBM = quandl.get("EOD/IBM")
IBM.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2013-09-03,183.63,184.32,182.51,183.96,3487200.0,0.0,1.0,147.900758,148.456503,146.998678,148.166549,3487200.0
2013-09-04,183.58,184.19,182.31,183.13,2597900.0,0.0,1.0,147.860486,148.351797,146.837593,147.498044,2597900.0
2013-09-05,183.35,185.00,183.07,184.15,2867600.0,0.0,1.0,147.675238,149.004194,147.449718,148.319580,2867600.0
2013-09-06,184.65,184.99,182.65,183.03,2903500.0,0.0,1.0,148.722294,148.996140,147.111438,147.417501,2903500.0
2013-09-09,183.68,185.49,183.31,184.98,3017200.0,0.0,1.0,147.941029,149.398854,147.643021,148.988086,3017200.0


In [6]:
IBM_sample = IBM['2015':"2017"].dropna()

In [7]:
IBM_market = IBM_sample['Close'].pct_change().dropna() * 100

## Computing $\beta$ value

The Beta of an investment is the volatility of the stock, in relation to the market. 

$\beta = 1$ If the stock moves with the market (the market itself has, by definition, a Beta of 1.0). 

$\beta < 1$ Stocks with low Beta values have less risk. 

$\beta < 1$ High Beta values have high risk.

To compute Beta, the equation is:

$\beta = \frac{cov(a, m)}{var(m)}$

Where $var(m)$ is the variance of the market, and $cov(a, m)$ is the covariance between the market and the given asset, in this case IBM.

In [8]:
market_variance = market["2015":"2017"].var()
print(f"Market Variance: {market_variance}")

Market Variance: 0.616294599585142


In [9]:
stock_cov = market.corr(IBM_market)

print(f"Stock Covariance: {stock_cov}")

Stock Covariance: 0.6086666731536453


In [10]:
beta = stock_cov / market_variance

print(f"Beta: {beta}")

Beta: 0.9876229218353829


$\beta = 1$ which means that the stock moves according to the market

## CAPM - Capital Asset Pricing Model

The CAPM model is a widely used model to determine expected returns for an asset and the impact of systematic risk. 

Systematic risk is the risk inherent to the entire market or market segment. Systematic risk, also known as “undiversifiable risk,” “volatility” or “market risk,” affects the overall market, not just a particular stock or industry. This type of risk is both unpredictable and impossible to completely avoid.

One method for calculating systematic risk is to measure the correlation between a stock and the market in general.

The CAPM model is:


$ CAPM = R_f + \beta (\bar{R_m} - R_f)$

Where:

* $CAPM$ is the expected return of the investment
* $R_f$ is the risk free rate (The risk-free rate of return is the theoretical rate of return of an investment with zero risk)
* $\beta$ is the Beta of the investment (calculated above)
* $\bar{R_m}$ is the expected return of the market

* Note that the value $(\bar{R_m} - R_f)$ is also known as the market risk premium.

In [11]:
e = ending_balance = sp500_sample.iloc[-1]["Last"]
s = starting_balance = sp500_sample.iloc[0]["Last"]
n = number_years = 3  # 2015, 2016, 2017
return_on_market = (e / s) ** (1 / n) - 1

print(f"Return on Market: {return_on_market}")

capm = .03  + beta * (return_on_market - .03)
print(f"CAPM: {capm}")


Return on Market: 0.09319535187362149
CAPM: 0.0924131780638412


## Conclusion

Capital Asset Pricing Model (CAPM) Summary
The CAPM uses the principles of Modern Portfolio Theory to determine if a security is fairly valued. It relies on assumptions about investor behaviors, risk and return distributions, and market fundamentals that don’t match reality. However, the underlying concepts of CAPM and the associated efficient frontier can help investors understand the relationship between expected risk and reward as they make better decisions about adding securities to a portfolio.

## References

- https://www.investopedia.com/terms/c/capm.asp - CAPM
- https://www.quandl.com/ - Stock datasets
- 